# Charlie Chaplin Time Machine

CIFAR-10 dataset from: https://www.cs.toronto.edu/~kriz/cifar.html

In [1]:
import numpy as np
import matplotlib as plt 
import cv2 as cv

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

cifar = unpickle("data/cifar-10/data_batch_1")
cifar.keys()
cifar[b'data'].shape

(10000, 3072)

### Plan of attack

>##### 1) Generate cifar-10 dataset, by making all images and B&W counter parts
>##### 2) Train U-Net
>##### 3) Eval Results

#### 1) Generate BW counter parts

In [12]:
import torch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, SubsetRandomSampler, random_split


(10000, 3072)